In [56]:
# public packages
import andes
import ams
import cvxpy as cp
import numpy as np
import gurobipy

# local packages
import anci_fun as af

# v0: make sure the modifed model based on 123 system is consistent with the original model in ams

# Para preparetion

In [57]:
# neural network parameters 
nn_name = 'trained_model.pth'
nn_weights, nn_bias, nn_diam = af.extract_nn_para(nn_name) # three list

# nn_diam = [27, 512, 6]

In [58]:
# MD parameters
vsg_num = 8

lb_M = np.array([0, 0, 0, 0, 0, 0, 0, 0])
ub_M = np.array([6, 6, 6, 6, 6, 6, 6, 6])

lb_D = np.array([0, 0, 0, 0, 0, 0, 0, 0])
ub_D = np.array([4, 4, 4, 4, 4, 4, 4, 4])

In [59]:
sf_load = 1.0

# y = [rocof_max, fnadir, dtheta_max, busV_min, eig_max, eig_max_]
lb_y = np.array([-0.5, -0.6, -3.14, 0, -100, -100])
ub_y = np.array([ 0.5,  0.6,  3.14, 2,    0,    0])

# Load ams model

In [60]:
ssp = ams.load('ams_case123_REGCV1_v5.xlsx')

ssp.LDOPF.run()

ssp.LDOPF.om.mdl.solve()

LDOPF solved as optimal in 0.0633 seconds with exit code 0.


0.40000492888023714

In [61]:
ssp.LDOPF.om.vars

OrderedDict([('pg', Variable((11,))),
             ('pn', Variable((117,))),
             ('qg', Variable((11,))),
             ('qn', Variable((117,))),
             ('vsq', Variable((117,))),
             ('pl', Variable((116,))),
             ('ql', Variable((116,)))])

In [62]:
pg_ams = ssp.LDOPF.om.pg.value

pg_ams

array([0.02000001, 0.00000001, 0.00000002, 0.05448514, 0.07447952,
       0.03400588, 0.03400588, 0.03400588, 0.03400588, 0.03400588,
       0.03400588])

In [63]:
ssp.LDOPF.pg.get_idx()

['PV_1',
 'PV_10',
 'PV_11',
 'PV_2',
 'PV_3',
 'PV_4',
 'PV_5',
 'PV_6',
 'PV_7',
 'PV_8',
 'PV_9']

# Define new variables, constraints, and obj

In [64]:
# M and D

M = cp.Variable(vsg_num, name="M")
D = cp.Variable(vsg_num, name="D")

constraints_M = [M >= lb_M, M <= ub_M]
constraints_D = [D >= lb_D, D <= ub_D]

In [65]:
constriants_ams = [ssp.LDOPF.om.constrs[cname] for cname in ssp.LDOPF.om.constrs.keys()]

In [66]:
constraints = constriants_ams + constraints_M + constraints_D

# Formulate new problem

pg = ['PV_1',
 'PV_10',
 'PV_11',
 'PV_2',
 'PV_3',
 'PV_4',
 'PV_5',
 'PV_6',
 'PV_7',
 'PV_8',
 'PV_9']

 SG: PV_1, PV_2, PV_3 (slack bus)

 IBR: PV_4, PV_5, ..., PV_11

In [67]:
# Cost data

c2 = np.array([0.01, 0, 0, 0.01, 0.02, 0, 0, 0, 0, 0, 0]) # gen quadratic cost
c1 = np.array([  20, 0, 0,   20,   30, 0, 0, 0, 0, 0, 0]) # gen linear cost

d  = np.array([0, 0, 0, 0, 0, 0, 0, 0])                   # gen virtual inertia cost

pg = ssp.LDOPF.om.pg

pg_power = cp.power(pg, 2)

cost = cp.sum(c2 @ pg_power + c1 @ pg + d @ M)

obj = cp.Minimize(cost)

In [68]:
# Create the optimization problem
prob = cp.Problem(obj, constraints)


In [69]:
# Solve the optimization problem

# prob.solve(solver=cp.GUROBI, verbose=True)

prob.solve()


2.00066158718484

In [70]:
ssp.LDOPF.om.pg.value

array([0.02000853, 0.0341217 , 0.0341217 , 0.02000853, 0.04000935,
       0.0341217 , 0.0341217 , 0.0341217 , 0.0341217 , 0.0341217 ,
       0.0341217 ])

In [71]:
M.value

array([0., 0., 0., 0., 0., 0., 0., 0.])

# 

In [72]:
D.value

array([0., 0., 0., 0., 0., 0., 0., 0.])